In [1]:
# Importing all libraries.
from pylab import *
from netCDF4 import Dataset
%matplotlib inline
import os
import cmocean as cm
from trackeddy.tracking import *
from trackeddy.datastruct import *
from trackeddy.geometryfunc import *
from trackeddy.init import *
from trackeddy.physics import *
from trackeddy.plotfunc import *
import time

In [2]:
# Output data path
outputpath='/g/data/v45/akm157/model_output/mom/mom01v5_kds75/output306/'
# Import SSH values to python environment.
ncfile=Dataset(outputpath+'rregionsouthern_ocean_daily_eta_t.nc')
eta=ncfile.variables['eta_t'][:]*100
# Import geographic coordinates (Lon,Lat)
lon=ncfile.variables['xt_ocean_sub01'][:]
lat=ncfile.variables['yt_ocean_sub01'][:]

In [3]:
# Import SSH 10 yrs mean values to python environment.
ncfile=Dataset('/home/156/jm5970/notebooks/traceddy/data.input/meanssh_10yrs_AEXP.nc')
ssh_mean=squeeze(ncfile.variables['SSH_mean'][:])
# Import geographic coordinates (Lon,Lat)
lon=ncfile.variables['Longitude'][:]
lat=ncfile.variables['Latitude'][:]

In [4]:
#Area in indexes, probably in the future it will be added an option for lon - lat coords.
areamap=array([[0,len(lon)],[0,len(lat)]])

In [5]:
def scan_eddym(ssh,lon,lat,levels,date,areamap,mask='',destdir='',physics='',basemap=False,eddycenter='masscenter',checkgauss=True,diagnostics=False,plotdata=False,pprint=True):
    '''
    *************Scan Eddym***********
    Function to identify each eddy using closed contours,
    also this function checks if the elipse adjusted have
    a consistent eccentricity, vorticty and other parameters.
    Usage:
    ssh= Sea Surface Height in cm
    lon,lat=longitude and latitude of your grid.
    levels=where the code will find the closed contours.
    date=date in julian days
    areamap=Section of interest
    mask=Continent mask
    
    Example:
    ssh=Read netCDF4 data with mask or create a mask for your data
    lon=Import your longitude coordinates, if your grid is regular, you can use a linspace instead
    lat=Import your latitude coordinates (same as above).
    levels=List of the levels in which ones you want to find the eddies
    date=Date as Julian Days
    areamap=array([[0,len(lon)],[0,len(lat)]]) Array with the index of your area of interest.
    I used some auxilar functions, each one has his respective author.
    Author: Josue Martinez Moreno, 2017
    '''
    inittime=time.time()
    ellipse_path=[]
    contour_path=[]
    mayoraxis_eddy=[]
    minoraxis_eddy=[]
    shapedata=np.shape(ssh)
    if ssh is ma.masked:
        print('Invalid ssh data, must be masked')
        return
    if shapedata == [len(lat), len(lon)]:
        print('Invalid ssh data size, should be [length(lat) length(lon]')
        return
    if np.shape(areamap) == shapedata:
        if np.shape(areamap) == [1, 1] | len(areamap) != len(lat):
            print('Invalid areamap, using NaN for eddy surface area')
        return
    if len(levels)!= 2:
        print('Invalid len of levels, please use the function for multiple levels or use len(levels)==2')
        return
    #Saving mask for future post-processing.  
    
    if mask!='':
        ssh=np.ma.masked_array(ssh, mask)
    sshnan=ssh.filled(np.nan)

    #sshnan=ma.masked_array(okparm, mask=mask[0,:,:])
    #sshnan=sshnan.filled(nan)
    #Obtain the contours of a surface (contourf), this aproach is better than the contour.
    if len(np.shape(lon))== 1 and len(np.shape(lat)) == 1:
        Lon,Lat=np.meshgrid(lon,lat)
    else:
        Lon,Lat=lon,lat
    
    min_x=Lon[0,0]
    min_y=Lat[0,0]
    max_x=Lon[-1,-1]
    max_y=Lat[-1,-1]
    
    if basemap==True:
        fig, ax = plt.subplots(figsize=(10,10))
        m = Basemap(projection='ortho',lat_0=-90,lon_0=-100,resolution='c')
        m.drawcoastlines()
        m.fillcontinents(color='black',lake_color='aqua')
        m.drawmeridians(np.arange(0,360,30),labels=[1,1,0,0],fontsize=10)
        lonm,latm=m(Lon,Lat)
    
        if len(shapedata)==3:
            m.contourf(lonm[areamap[1,0]:areamap[1,1],areamap[0,0]:areamap[0,1]],\
                       latm[areamap[1,0]:areamap[1,1],areamap[0,0]:areamap[0,1]],\
                    sshnan[date,areamap[1,0]:areamap[1,1],areamap[0,0]:areamap[0,1]],levels=levels)
            plt.show()

        else:
            m.contourf(lonm[areamap[1,0]:areamap[1,1],areamap[0,0]:areamap[0,1]],\
                       latm[areamap[1,0]:areamap[1,1],areamap[0,0]:areamap[0,1]],\
                    sshnan[areamap[1,0]:areamap[1,1],areamap[0,0]:areamap[0,1]],levels=levels)
            plt.show()
            
    if len(shapedata)==3:
        CS=plt.contourf(lon[areamap[0,0]:areamap[0,1]],lat[areamap[1,0]:areamap[1,1]],\
                sshnan[date,areamap[1,0]:areamap[1,1],areamap[0,0]:areamap[0,1]],levels=levels)
    else:
        CS=plt.contourf(lon[areamap[0,0]:areamap[0,1]],lat[areamap[1,0]:areamap[1,1]],\
                sshnan[areamap[1,0]:areamap[1,1],areamap[0,0]:areamap[0,1]],levels=levels)
    # Close the contour plot.
    plt.close()
    CONTS=CS.allsegs[:][:]
    #Loop in contours of the levels defined.
    total_contours=0
    eddyn=0
    print('Init time:',time.time()-inittime)
    looptime=time.time()
    for ii in range(0,np.shape(CONTS)[0]):
        CONTSlvls=CONTS[ii]
        for jj in range(0,np.shape(CONTSlvls)[0]):
            CONTeach=CONTSlvls[jj]
            if (len(CONTeach[:,0]) | len(CONTeach[:,1])) <= 10:
                xx=np.nan
                yy=np.nan
                center=[np.nan,np.nan]
                check=False
            else:
                ellipse,status=fit_ellipse(CONTeach[:,0],CONTeach[:,1],diagnostics=diagnostics)
                if status==True:
                    center = [ellipse['X0_in'],ellipse['Y0_in']]
                    phi = ellipse['phi']
                    axes = [ellipse['a'],ellipse['b']]
                    R = np.arange(0,2.1*np.pi, 0.1)
                    a,b = axes
                    #Ellipse coordinates.
                    xx = ellipse['ellipse'][0]
                    yy = ellipse['ellipse'][1]
                    
                    mayoraxis = ellipse['majoraxis']
                    minoraxis = ellipse['minoraxis']
                    
                    #Area of Contours (contarea) and ellipse (ellipsarea)
                    contarea=PolyArea(CONTeach[:,0],CONTeach[:,1])
                    ellipsarea=PolyArea(xx,yy)
                    
                    # Linear Eccentricity check
                    eccen=eccentricity(a,b)
                    #Record and check how many grid points have land or masked values
                    landcount=0
                    for ii in range(0,len(CONTeach[:,0])):
                        idxcheck,idycheck=find2d(lon,lat,CONTeach[ii,0],CONTeach[ii,1])
                        idxelipcheck,idyelipcheck=find2d(lon,lat,center[0],center[1])
                        if len(shapedata)==3:
                            if sshnan[date,idycheck,idxcheck]==np.nan:
                                landcount=countzeros+1
                        else:
                            if sshnan[idycheck,idxcheck]==np.nan:
                                landcount=countzeros+1
                    if landcount>=len(CONTeach[:,0])/2:
                        #print 'Thisone is land'
                        check=False
                    else:
                        if contarea>ellipsarea:
                            #if contarea/1.5>ellipsarea:
                            #    check=False
                            if eccen<1 and eccen>0:
                                if ellipsarea < 200 and contarea < 200:
                                    if checkgauss==True:
                                        checke=False
                                        checkM=False
                                        checkm=False
                                        
                                        #tice=time.time()
                                        ellipseadjust,checke=ellipsoidfit(CONTeach[:,1],ellipse['ellipse']\
                                                                             [1],diagnostics=diagnostics)
                                        #print('Time elapsed ellipsoidfit:',str(time.time()-tice))
                                        
                                        if checke==True:
                                            #ticg=time.time()
                                            if len(shapedata)==3:
                                                profile,checkM=extractprofeddy(mayoraxis,sshnan[date,:,:],lon,lat,50,\
                                                                  gaus='One',kind='linear',diagnostics=False)
                                                if checkM==True:
                                                    profile,checkm=extractprofeddy(minoraxis,sshnan[date,:,:],lon,lat,50,\
                                                                      gaus='One',kind='linear',diagnostics=False)
                                            else:
                                                profile,checkM=extractprofeddy(mayoraxis,sshnan[:,:],lon,lat,50,\
                                                                  gaus='One',kind='linear',diagnostics=False)
                                                if checkM==True:
                                                    profile,checkm=extractprofeddy(minoraxis,sshnan[:,:],lon,lat,50,\
                                                                      gaus='One',kind='linear',diagnostics=False)
                                        #print('Time elapsed ellipsoidfit:',str(time.time()-ticg))
                                        if checkM==True and  checke==True and checkm==True: 
                                            check=True
                                        else:
                                            check=False
                                else:
                                    check=False
                            else:
                                check=False
                        elif contarea<ellipsarea:
                            #if contarea<ellipsarea/1.5:
                                #print 'Removing contour, thisone is really overestimate'
                            #    check=False
                            #elif eccen<0.95 and eccen>0.4:
                            if eccen<1 and eccen>0:
                                if ellipsarea < 200 and contarea<200:
                                    if checkgauss==True:
                                        checke=False
                                        checkM=False
                                        checkm=False
                                        
                                        tic=time.time()
                                        ellipseadjust,checke=ellipsoidfit(CONTeach[:,1],ellipse['ellipse']\
                                                                             [1],diagnostics=diagnostics)
                                        #print('Time elapsed ellipsoidfit:',str(time.time()-tic))
                                        
                                        if checke==True:
                                            tic=time.time()
                                            if len(shapedata)==3:
                                                profile,checkM=extractprofeddy(mayoraxis,sshnan[date,:,:],lon,lat,50,\
                                                                  gaus='One',kind='linear',diagnostics=False)
                                                if checkM==True:
                                                    profile,checkm=extractprofeddy(minoraxis,sshnan[date,:,:],lon,lat,50,\
                                                                      gaus='One',kind='linear',diagnostics=False)
                                            else:
                                                profile,checkM=extractprofeddy(mayoraxis,sshnan[:,:],lon,lat,50,\
                                                                  gaus='One',kind='linear',diagnostics=False)
                                                if checkM==True:
                                                    profile,checkm=extractprofeddy(minoraxis,sshnan[:,:],lon,lat,50,\
                                                                      gaus='One',kind='linear',diagnostics=False)
                                        #print('Time elapsed ellipsoidfit:',str(time.time()-tic))
                                        if checkM==True and  checke==True and checkm==True: 
                                            check=True
                                        else:
                                            check=False
                                else:
                                    check=False
                            else:
                                #print 'Removing contour, thisone is really underestimate'
                                check=False
                        else:
                            check=False
                            
                    if diagnostics == True and  check == True:
                        print("Ellipse parameters")
                        print("Ellipse center = ",  center)
                        print("angle of rotation = ",  phi)
                        print("axes (a,b) = ", axes)
                        print("Eccentricity = ",eccen)
                        print("Area (cont,ellips) = ",contarea,ellipsarea)
                        print("Ellipse adjust = ",ellipseadjust)
                    if check==True:# or check==False:
                        ellipse_path.append([xx,yy])
                        contour_path.append([CONTeach[:,0],CONTeach[:,1]])
                        mayoraxis_eddy.append([mayoraxis[0],mayoraxis[1]])
                        minoraxis_eddy.append([minoraxis[0],minoraxis[1]])
                        #Switch from the ellipse center to the position of the maximum value inside de contour
                        if eddycenter == 'maximum':
                            center_eddy=contourmaxvalue(CONTeach[:,0],CONTeach[:,1],sshnan,lon,lat,levels,date)
                        elif eddycenter == 'masscenter':
                            center_eddy=centroidvalue(CONTeach[:,0],CONTeach[:,1],sshnan,lon,lat,levels,date)
                        if eddyn==0:
                            position_eddy=center_eddy
                            position_ellipse=center
                            total_eddy=eddyn
                            area=contarea
                            angle=phi
                            if CS.levels[0] > 0:
                                level=CS.levels[0]
                            else:
                                level=CS.levels[1]
                            levelprnt=level
                        else:
                            position_eddy=np.vstack((position_eddy,center_eddy))
                            position_ellipse=np.vstack((position_ellipse,center))
                            total_eddy=np.vstack((total_eddy,eddyn))
                            area=np.vstack((area,contarea))
                            angle=np.vstack((angle,phi))
                            
                            if CS.levels[0] > 0:
                                levelprnt=CS.levels[0]
                                level=np.vstack((level,levelprnt))
                            else:
                                levelprnt=CS.levels[1]
                                level=np.vstack((level,levelprnt))
                        
                        eddyn=eddyn+1
                    if diagnostics == True and plotdata == True:
                        f, (ax1, ax2) = plt.subplots(1, 2,figsize=(13, 6))
                        if len(shapedata)==3:
                            ax1.contourf(lon[areamap[0,0]:areamap[0,1]],lat[areamap[1,0]:areamap[1,1]],\
                                ssh[date,areamap[1,0]:areamap[1,1],areamap[0,0]:areamap[0,1]])
                            cc=ax2.pcolormesh(lon[areamap[0,0]:areamap[0,1]],lat[areamap[1,0]:areamap[1,1]],\
                                ssh[date,areamap[1,0]:areamap[1,1],areamap[0,0]:areamap[0,1]],\
                                          vmin=ssh[date,areamap[1,0]:areamap[1,1],areamap[0,0]:areamap[0,1]].min()\
                                          ,vmax=ssh[date,areamap[1,0]:areamap[1,1],areamap[0,0]:areamap[0,1]].max())
                            cca=ax2.contour(lon[areamap[0,0]:areamap[0,1]],lat[areamap[1,0]:areamap[1,1]],\
                                ssh[date,areamap[1,0]:areamap[1,1],areamap[0,0]:areamap[0,1]],levels=levels,cmap='jet')
                            ax2.clabel(cca, fontsize=9, inline=1)
                        else:
                            cca=ax1.contourf(lon[areamap[0,0]:areamap[0,1]],lat[areamap[1,0]:areamap[1,1]],\
                            sshnan[areamap[1,0]:areamap[1,1],areamap[0,0]:areamap[0,1]],levels=levels)
                            ax1.plot(CONTeach[:,0],CONTeach[:,1],'-r')
                            ax2.plot(CONTeach[:,0],CONTeach[:,1],'-r')
                            ax2.pcolormesh(lon[areamap[0,0]:areamap[0,1]],lat[areamap[1,0]:areamap[1,1]],\
                            sshnan[areamap[1,0]:areamap[1,1],areamap[0,0]:areamap[0,1]],vmin=-20,vmax=20)
                            plt.show()
                            f, (ax1, ax2) = plt.subplots(1, 2,figsize=(13, 6))
                            ax1.contourf(lon[areamap[0,0]:areamap[0,1]],lat[areamap[1,0]:areamap[1,1]],\
                                    ssh[areamap[1,0]:areamap[1,1],areamap[0,0]:areamap[0,1]])
                            cc=ax2.pcolormesh(lon[areamap[0,0]:areamap[0,1]],lat[areamap[1,0]:areamap[1,1]],\
                                    ssh[areamap[1,0]:areamap[1,1],areamap[0,0]:areamap[0,1]],\
                                              vmin=ssh[areamap[1,0]:areamap[1,1],areamap[0,0]:areamap[0,1]].min()\
                                              ,vmax=ssh[areamap[1,0]:areamap[1,1],areamap[0,0]:areamap[0,1]].max())
                            cca=ax2.contour(lon[areamap[0,0]:areamap[0,1]],lat[areamap[1,0]:areamap[1,1]],\
                                    ssh[areamap[1,0]:areamap[1,1],areamap[0,0]:areamap[0,1]],levels=levels,cmap='jet')
                            ax2.clabel(cca, fontsize=9, inline=1)
                        ax1.plot(CONTeach[:,0],CONTeach[:,1],'*r')
                        ax1.plot(xx,yy,'-b')
                        ax1.plot(center[0],center[1],'ob')
                        f.subplots_adjust(right=0.8)
                        cbar_ax = f.add_axes([0.85, 0.15, 0.05, 0.7])
                        f.colorbar(cc, cax=cbar_ax)
                        ax2.plot(CONTeach[:,0],CONTeach[:,1],'-r')
                        ax2.plot(xx,yy,'-b')
                        ax2.plot(center[0],center[1],'ob')
                        ax2.plot(lon[idxelipcheck],lat[idyelipcheck],'om')
                        ax2.set_ylim([CONTeach[:,1].min(),CONTeach[:,1].max()])
                        ax2.set_xlim([CONTeach[:,0].min(),CONTeach[:,0].max()])
                        plt.show()
                        plt.close()
                total_contours=total_contours+1
            if pprint==True:
                string='Total of contours was: %d - Total of eddies: %d - Level: %.1f   ' % (total_contours,eddyn,levelprnt)
                pt =Printer(); pt.printtextoneline(string)
        position_eddy=np.array(position_eddy)
        position_ellipse=np.array(position_ellipse)
        level=np.array(level)
        mayoraxis_eddy=np.array(mayoraxis_eddy)
        minoraxis_eddy=np.array(minoraxis_eddy)
        eddys=dict_eddym(contour_path,ellipse_path,position_eddy,position_ellipse,mayoraxis_eddy,minoraxis_eddy,\
                         area,angle,total_eddy,level)
        #if destdir!='':
        #    save_data(destdir+'day'+str(date)+'_one_step_cont'+str(total_contours)+'.dat', variable)
    print('Loop time:',time.time()-inittime)    
    return eddys

In [6]:
import time

def analyseddyzt(data,x,y,t0,t1,tstep,maxlevel,minlevel,dzlevel,data_meant='',areamap='',mask='',destdir='',physics='',eddycenter='masscenter',checkgauss=True,diagnostics=False,plotdata=False,pprint=False):
    '''
    *************Analys eddy in z and t ***********
    Function to identify each eddy using closed contours, 
    moving in time and contour levels
    Usage:
    
    Example:

    Author: Josue Martinez Moreno, 2017
    '''
    if len(np.shape(data))<3:
        print('If you whant to analyze in time the data need to be 3d [i.e. data(t,x,y)]')
        #return
    if areamap=='':
        areamap=np.array([[0,len(x)],[0,len(y)]])
    if mask == '':
        if len(np.shape(data))<3:
            mask=ma.getmask(data[:,:])
        else:
            mask=ma.getmask(data[0,:,:])
        
    pp =  Printer(); 
    for ii in range(t0,t1,tstep):
        levellist=np.arange(minlevel,maxlevel+dzlevel,dzlevel)
        farlevel=levellist[0]
        if abs(levellist)[0]<abs(levellist)[-1]:
            levellist=np.flipud(levellist)
            farlevel=levellist[0]
        if data_meant=='':
            print('Be sure the data is an anomaly')
            dataanomaly=data[ii,:,:]
        else:
            dataanomaly=data[ii,:,:]-data_meant
        for ll in levellist:
            if minlevel<0 and maxlevel<0:
                levels=[-500,ll]
            elif minlevel>0 and maxlevel>0:
                levels=[ll,500]
            tic=time.time()
            eddies=scan_eddym(dataanomaly,x,y,levels,ii,areamap,mask=mask,destdir=destdir\
                          ,physics=physics,eddycenter=eddycenter,checkgauss=checkgauss\
                          ,diagnostics=diagnostics,plotdata=plotdata,pprint=pprint)
            print('ellapse identification:',time.time()-tic)
            if ll == farlevel:
                eddz = dict_eddyz(ii,ll,farlevel,eddies,diagnostics=diagnostics)
            else:
                tic=time.time()
                eddz = dict_eddyz(ii,ll,farlevel,eddies,eddz,diagnostics=diagnostics)
                print('ellapse dz:',time.time()-tic)
        if ii==0:
            eddytd=dict_eddyt(ii,eddz)
        else:
            tic=time.time()
            eddytd=dict_eddyt(ii,eddz,eddytd) 
            print('ellapse dt:',time.time()-tic)
        pp.timepercentprint(t0,t1,tstep,ii)
    return eddytd

In [ ]:
import time

def dict_eddyt(ts,eddys,eddydt=''):
    '''
    Function to create a dictionary with all the eddies.
    All the keys have the following form:
    {eddyn_0:{neddy,time,position,ellipse,contour},eddyn_1:{neddy,time,position,ellipse,contour}}
    Usage:
    
    '''

    #print('eddys',eddys['MinorAxis'])
    
    if ts==0 or eddydt=='':
        #Define the variable
        eddydt={'eddyn_'+str(eddys['EddyN'][0][0]):{'neddy':eddys['EddyN'][0],'time':ts,'position':eddys['Position'][0],\
                'area':eddys['Area'][0],'ellipse':[eddys['Ellipse'][0]],'contour':[eddys['Contour'][0]],\
                'angle':eddys['Angle'][0],'position_eddy':eddys['PositionEllipse'][0],'level':eddys['Level'][0],\
                'majoraxis':eddys['MajorAxis'][0],'minoraxis':eddys['MinorAxis'][0],'timetracking':True}}
        #Append all the new data
        for nn in range(1,len(eddys['EddyN'])):
            eddydt['eddyn_'+str(eddys['EddyN'][nn][0])]={'neddy':eddys['EddyN'][nn],'time':ts,'position':eddys['Position'][nn],\
                    'area':eddys['Area'][nn],'ellipse':[eddys['Ellipse'][nn]],'contour':[eddys['Contour'][nn]],\
                    'angle':eddys['Angle'][nn],'position_eddy':eddys['PositionEllipse'][nn],'level':eddys['Level'][nn],\
                    'majoraxis':eddys['MajorAxis'][nn],'minoraxis':eddys['MinorAxis'][nn],'timetracking':True}
    else: 
        checklist=[]
        checklist1=[]
        ceddies=[]
        ### I should be able to parallelize this loop, It's the one that is taking more time 
        #tic=time.time()
        for key, value in list(eddydt.items()):
            if value['timetracking']==False:
                print('No tracking this eddy.')
            else:
                minoraxis= value['minoraxis']
                #print('test',np.shape(minoraxis))
                majoraxis= value['majoraxis']
                number=value['neddy']
                times=value['time']
                position=value['position']
                position_eddy=value['position_eddy']
                ellipse=value['ellipse']
                contour=value['contour']
                level= value['level']
                if isinstance(number, np.float64) or isinstance(number, np.int64):
                    number = number
                else: 
                    number = number[0]
            
                for nn in range(0,len(eddys['EddyN'])):
                    maxlon=eddys['Contour'][nn][0].max()
                    maxlone=eddys['Ellipse'][nn][0].max()
                    maxlat=eddys['Contour'][nn][1].max()
                    maxlate=eddys['Ellipse'][nn][1].max()
                    minlon=eddys['Contour'][nn][0].min()
                    minlone=eddys['Ellipse'][nn][0].min()
                    minlat=eddys['Contour'][nn][1].min()
                    minlate=eddys['Ellipse'][nn][1].min()
                    area=eddys['Area'][nn]
                    angle=eddys['Angle'][nn]                
                    if len(np.shape(value['position']))<2:
                        eddyxt0=value['position'][0]
                        eddyyt0=value['position'][1]
                        areae=value['area']
                        anglee=value['angle']
                        timee=value['time']
                    else:
                        eddyxt0=value['position'][-1,0]
                        eddyyt0=value['position'][-1,1]
                        areae=value['area'][-1]
                        anglee=value['angle'][-1]
                        timee=value['time'][-1]
                    eddyxt1=eddys['Position'][nn][0]
                    eddyyt1=eddys['Position'][nn][1]
                
                    ################################################################
                    #### CHECK WHY THE MINORAXIS Probable bug somewhere else.#######
                    ################################################################
                
                    ################################################################
                    ######## Add an analysis depending the maximum z level #########
                    ################################################################
                    #print(len(np.shape(minoraxis)),np.shape(minoraxis),np.shape(eddys['MinorAxis'][nn]))
                    if len(np.shape(minoraxis))==3:
                        minoraxis=np.squeeze(minoraxis)
                    if len(np.shape(majoraxis))==3:
                        majoraxis=np.squeeze(majoraxis)
                
                    if (eddyxt1<=maxlon and eddyxt1>=minlon and eddyyt1<=maxlat and eddyyt1>=minlat) and\
                        (eddyxt0<=maxlon and eddyxt0>=minlon and eddyyt0<=maxlat and eddyyt0>=minlat) and\
                        int(timee)!=ts and (ts-int(timee))<5:
                        # and (areae<=area+area/4 and areae>=area-area/4):
                       
                        #print(len(np.shape(minoraxis)),np.shape(minoraxis),np.shape(eddys['MinorAxis'][nn]))
                        eddydt['eddyn_'+str(number)]={'neddy':number,'time':np.vstack((value['time'],ts)),\
                                            'position':np.vstack((position,eddys['Position'][nn])),\
                                            'area':np.vstack((areae,area)),'angle':np.vstack((value['angle'],angle)),\
                                            'ellipse':ellipse+[eddys['Ellipse'][nn]],\
                                            'contour':contour+[eddys['Contour'][nn]],\
                                            'position_eddy':np.vstack((position_eddy,eddys['PositionEllipse'][nn])),\
                                            'level':np.vstack((level,eddys['Level'][nn])),\
                                            'minoraxis':np.vstack((minoraxis,np.squeeze(eddys['MinorAxis'][nn]))),\
                                            'majoraxis':np.vstack((majoraxis,np.squeeze(eddys['MajorAxis'][nn]))),\
                                            'timetracking':True}
                    #print(np.shape(np.vstack((minoraxis,eddys['MinorAxis'][nn]))),np.shape(eddys['MinorAxis'][nn]))
                        checklist1.append(eddys['EddyN'][nn][0])
                        checklist.append(number)
                    elif (eddyxt1<=maxlon and eddyxt1>=minlon and eddyyt1<=maxlat and eddyyt1>=minlat) and\
                        (eddyxt0<=maxlon and eddyxt0>=minlon and eddyyt0<=maxlat and eddyyt0>=minlat) and\
                        int(timee)!=ts and (ts-int(timee))>=5:
                        print('Add Removal')
                        eddydt['eddyn_'+str(number)]['timetracking']=False
                    
                ceddies.append(number)
                
        #print('ellapse dt checking existing eddies:',time.time()-tic)
        counter=0
        checklist2=[]
        checklist3=[]
        #tic=time.time()
        for key, value in list(eddydt.items()):
            if value['timetracking']==True:
                number=value['neddy']
                if isinstance(number, np.float64) or isinstance(number, np.int64):
                    number = number
                else: 
                    number = number[0]
                for nn in range(0,len(eddys['EddyN'])):
                    number1=eddys['EddyN'][nn]
                    
                    #print((number!=checklist),(number1!=checklist1))
                    if (number!=checklist).all() and (number1!=checklist1).all() and (number!=checklist2).all()\
                        and (number1!=checklist3).all():
                        checklist2.append(number)
                        checklist3.append(number1)
                        counter=counter+1
                        #print '*****New Eddy*****'
                        #print(number,checklist)
                        #print('max value number',np.max(ceddies))
                        number=np.max(ceddies)+counter
                        if isinstance(number, np.float64) or isinstance(number, np.int64):
                            number = number
                            #print('numberint',str(number),eddys['EddyN'][nn][0])
                        else: 
                            number = eddys['EddyN'][nn][0]
                        #   print('numberlist',str(number))
                
                        #print(eddydt[:]['neddy'])
                        #print(number)
                        eddydt['eddyn_'+str(number)]={'neddy':number,'time':ts,\
                                        'position':eddys['Position'][nn],'area':eddys['Area'][nn],'angle':eddys['Angle'][nn],\
                                        'position_eddy':eddys['PositionEllipse'][nn],'ellipse':[eddys['Ellipse'][nn]],\
                                        'contour':[eddys['Contour'][nn]],'level':eddys['Level'][nn],\
                                        'minoraxis':eddys['MinorAxis'][nn],'majoraxis':eddys['MajorAxis'][nn],\
                                        'timetracking':True}
                        
        #print('ellapse dt checking NON existing eddies:',time.time()-tic)
    return eddydt

In [ ]:
eddytd=analyseddyzt(eta,lon,lat,0,10,1,25,5,5,data_meant=ssh_mean,areamap=areamap,mask=''\
                     ,destdir='',physics='',diagnostics=False,pprint=False)

Init time: 0.345780611038208
Loop time: 41.590575218200684
ellapse identification: 41.591126918792725
Init time: 0.40903234481811523
Loop time: 62.69270992279053
ellapse identification: 62.69289207458496
ellapse dz: 0.020142316818237305
Init time: 0.36534571647644043
Loop time: 92.4699535369873
ellapse identification: 92.47016286849976
ellapse dz: 0.0350794792175293
Init time: 0.3466942310333252
Loop time: 146.42438507080078
ellapse identification: 146.42463088035583
ellapse dz: 0.08161401748657227
Init time: 0.37183141708374023
Loop time: 182.6118037700653
ellapse identification: 182.61206436157227
ellapse dz: 0.15037751197814941
 0% [>]10% Time Elapsed: 526 s  Init time: 0.3397367000579834
Loop time: 43.53916788101196
ellapse identification: 43.53940510749817
Init time: 0.335573673248291
Loop time: 60.507160663604736
ellapse identification: 60.5073766708374
ellapse dz: 0.02057814598083496
Init time: 0.33803367614746094
Loop time: 93.7032241821289
ellapse identification: 93.7034597396

Add Removal
Add Removal
Add Removal
Add Removal
Add Removal
Add Removal
ellapse dt checking existing eddies: 31.08768582344055
ellapse dt checking NON existing eddies: 74.39826035499573
ellapse dt: 105.4870707988739
 0% [=======>]80% Time Elapsed: 5190 s  Init time: 0.46268463134765625
Loop time: 58.43780303001404
ellapse identification: 58.43803334236145
Init time: 0.4164741039276123
Loop time: 94.54810190200806
ellapse identification: 94.54829788208008
ellapse dz: 0.016640424728393555
Init time: 0.34140682220458984
Loop time: 157.02235007286072
ellapse identification: 157.02255201339722
ellapse dz: 0.036090850830078125
Init time: 0.3954024314880371
Loop time: 174.33304023742676
ellapse identification: 174.33325338363647
ellapse dz: 0.07212424278259277
Init time: 0.33356475830078125
Loop time: 281.24806475639343
ellapse identification: 281.2482662200928
ellapse dz: 0.13139724731445312
Add Removal
No tracking this eddy.
No tracking this eddy.
No tracking this eddy.
No tracking this edd

In [ ]:
from joblib import Parallel, delayed
import multiprocessing
import tempfile

def neweddystruct(ts,value,eddys,eddydt):
    checklist=[]
    checklist1=[]
    ceddies=[]
    minoraxis= value['minoraxis']
    #print('test',np.shape(minoraxis))
    majoraxis= value['majoraxis']
    number=value['neddy']
    time=value['time']
    position=value['position']
    position_eddy=value['position_eddy']
    ellipse=value['ellipse']
    contour=value['contour']
    level= value['level']
    if isinstance(number, np.float64) or isinstance(number, np.int64):
        number = number
    else: 
        number = number[0]
    
    for nn in range(0,len(eddys['EddyN'])):
        maxlon=eddys['Contour'][nn][0].max()
        maxlone=eddys['Ellipse'][nn][0].max()
        maxlat=eddys['Contour'][nn][1].max()
        maxlate=eddys['Ellipse'][nn][1].max()
        minlon=eddys['Contour'][nn][0].min()
        minlone=eddys['Ellipse'][nn][0].min()
        minlat=eddys['Contour'][nn][1].min()
        minlate=eddys['Ellipse'][nn][1].min()
        area=eddys['Area'][nn]
        angle=eddys['Angle'][nn]                
        if len(np.shape(value['position']))<2:
            eddyxt0=value['position'][0]
            eddyyt0=value['position'][1]
            areae=value['area']
            anglee=value['angle']
            timee=value['time']
        else:
            eddyxt0=value['position'][-1,0]
            eddyyt0=value['position'][-1,1]
            areae=value['area'][-1]
            anglee=value['angle'][-1]
            timee=value['time'][-1]
        eddyxt1=eddys['Position'][nn][0]
        eddyyt1=eddys['Position'][nn][1]
                
        ################################################################
        #### CHECK WHY THE MINORAXIS Probable bug somewhere else.#######
        ################################################################
        
        ################################################################
        ######## Add an analysis depending the maximum z level #########
        ################################################################
        #print(len(np.shape(minoraxis)),np.shape(minoraxis),np.shape(eddys['MinorAxis'][nn]))
        if len(np.shape(minoraxis))==3:
            minoraxis=np.squeeze(minoraxis)
        if len(np.shape(majoraxis))==3:
            majoraxis=np.squeeze(majoraxis)
                
        if (eddyxt1<=maxlon and eddyxt1>=minlon and eddyyt1<=maxlat and eddyyt1>=minlat) and\
            (eddyxt0<=maxlon and eddyxt0>=minlon and eddyyt0<=maxlat and eddyyt0>=minlat) and\
            int(timee)!=ts and (ts-int(timee))<5:# and (areae<=area+area/4 and areae>=area-area/4):
                       
            #print(len(np.shape(minoraxis)),np.shape(minoraxis),np.shape(eddys['MinorAxis'][nn]))
            eddydt['eddyn_'+str(number)]={'neddy':number,'time':np.vstack((value['time'],ts)),\
                                        'position':np.vstack((position,eddys['Position'][nn])),\
                                        'area':np.vstack((areae,area)),'angle':np.vstack((value['angle'],angle)),\
                                        'ellipse':ellipse+[eddys['Ellipse'][nn]],\
                                        'contour':contour+[eddys['Contour'][nn]],\
                                        'position_eddy':np.vstack((position_eddy,eddys['PositionEllipse'][nn])),\
                                        'level':np.vstack((level,eddys['Level'][nn])),\
                                        'minoraxis':np.vstack((minoraxis,np.squeeze(eddys['MinorAxis'][nn]))),\
                                        'majoraxis':np.vstack((majoraxis,np.squeeze(eddys['MajorAxis'][nn])))}
            #print(np.shape(np.vstack((minoraxis,eddys['MinorAxis'][nn]))),np.shape(eddys['MinorAxis'][nn]))
            checklist1.append(eddys['EddyN'][nn][0])
            checklist.append(number)
    ceddies.append(number)
    return eddydt,checklist,checklist1,ceddies

def dict_eddyt(ts,eddys,eddydt=''):
    '''
    Function to create a dictionary with all the eddies.
    All the keys have the following form:
    {eddyn_0:{neddy,time,position,ellipse,contour},eddyn_1:{neddy,time,position,ellipse,contour}}
    Usage:
    
    '''

    #print('eddys',eddys['MinorAxis'])

    if ts==0 or eddydt=='':
        #Define the variable
        eddydt={'eddyn_'+str(eddys['EddyN'][0][0]):{'neddy':eddys['EddyN'][0],'time':ts,'position':eddys['Position'][0],\
                'area':eddys['Area'][0],'ellipse':[eddys['Ellipse'][0]],'contour':[eddys['Contour'][0]],\
                'angle':eddys['Angle'][0],'position_eddy':eddys['PositionEllipse'][0],'level':eddys['Level'][0],\
                'majoraxis':eddys['MajorAxis'][0],'minoraxis':eddys['MinorAxis'][0]}}
        #Append all the new data
        for nn in range(1,len(eddys['EddyN'])):
            eddydt['eddyn_'+str(eddys['EddyN'][nn][0])]={'neddy':eddys['EddyN'][nn],'time':ts,'position':eddys['Position'][nn],\
                    'area':eddys['Area'][nn],'ellipse':[eddys['Ellipse'][nn]],'contour':[eddys['Contour'][nn]],\
                    'angle':eddys['Angle'][nn],'position_eddy':eddys['PositionEllipse'][nn],'level':eddys['Level'][nn],\
                    'majoraxis':eddys['MajorAxis'][nn],'minoraxis':eddys['MinorAxis'][nn]}
    else: 
        
        #folder = tempfile.mkdtemp()
        #eddies_name = os.path.join(folder, 'eddies')
        #sums = np.memmap(eddies_name, dtype=eddydt.dtype,
        #                 shape=len(eddydt.items()), mode='w+')
    
        #dump(eddydt, eddies_name)
        #eddydt = load(eddies_name, mmap_mode='r')
        #print(eddydt)
        
        ### I should be able to parallelize this loop, It's the one that is taking more time 
        tic=time.time()
        #num_cores = multiprocessing.cpu_count()
        pool = multiprocessing.Pool(processes=4)
        results = [pool.apply(neweddystruct, args=(ts,value,eddys,eddydt)) for key, value in list(eddydt.items())]
        #eddydt,checklist,checklist1,ceddies = Parallel(n_jobs=num_cores)(delayed(neweddystruct)(ts,value,eddys) \
        #                                                                 for key, value in list(eddydt.items()))
        
            
        print('ellapse dt checking existing eddies:',time.time()-tic)
        counter=0
        checklist2=[]
        checklist3=[]
        tic=time.time()
        for key, value in list(eddydt.items()):
            number=value['neddy']
            if isinstance(number, np.float64) or isinstance(number, np.int64):
                number = number
            else: 
                number = number[0]
            for nn in range(0,len(eddys['EddyN'])):
                number1=eddys['EddyN'][nn]
                
                #print((number!=checklist),(number1!=checklist1))
                if (number!=checklist).all() and (number1!=checklist1).all() and (number!=checklist2).all() and (number1!=checklist3).all():
                    checklist2.append(number)
                    checklist3.append(number1)
                    counter=counter+1
                    #print '*****New Eddy*****'
                    #print(number,checklist)
                    #print('max value number',np.max(ceddies))
                    number=np.max(ceddies)+counter
                    if isinstance(number, np.float64) or isinstance(number, np.int64):
                        number = number
                        #print('numberint',str(number),eddys['EddyN'][nn][0])
                    else: 
                        number = eddys['EddyN'][nn][0]
                    #   print('numberlist',str(number))
                
                    #print(eddydt[:]['neddy'])
                    #print(number)
                    eddydt['eddyn_'+str(number)]={'neddy':number,'time':ts,\
                                        'position':eddys['Position'][nn],'area':eddys['Area'][nn],'angle':eddys['Angle'][nn],\
                                        'position_eddy':eddys['PositionEllipse'][nn],'ellipse':[eddys['Ellipse'][nn]],\
                                        'contour':[eddys['Contour'][nn]],'level':eddys['Level'][nn],\
                                        'minoraxis':eddys['MinorAxis'][nn],'majoraxis':eddys['MajorAxis'][nn]}
                #print('New?',number)
        print('ellapse dt checking NON existing eddies:',time.time()-tic)
        #eddydt={'eddyn'+str(eddys['EddyN']):{'time':ts,'position':eddys['Position'],'ellipse':eddys['Ellipse']\
#                                    ,'contour':eddys['Contour']}}
    #print eddydt
    #print(counter)
    return eddydt


In [ ]:
eddytd=analyseddyzt(eta,lon,lat,0,10,1,25,5,5,data_meant=ssh_mean,areamap=areamap,mask=''\
                     ,destdir='',physics='',diagnostics=False,pprint=False)